<br> Radley Ciego </br>
<br> GTECH 73100 </br>
<br> Assignment 8: Pandas & Geopandas </br>

In [1]:
import pandas as pd
import geopandas as gpd

In [4]:
counties = pd.read_json("./gz_2010_us_050_00_5m.json")
counties.head()

,type,features
0,FeatureCollection,"{'type': 'Feature', 'properties': {'GEO_ID': '..."
1,FeatureCollection,"{'type': 'Feature', 'properties': {'GEO_ID': '..."
2,FeatureCollection,"{'type': 'Feature', 'properties': {'GEO_ID': '..."
3,FeatureCollection,"{'type': 'Feature', 'properties': {'GEO_ID': '..."
4,FeatureCollection,"{'type': 'Feature', 'properties': {'GEO_ID': '..."


## Task 1:
Find top n most common county names (n = 3 or 5, for example)

## Task 2:
Derive summary statistics for states, for example, number of counties, counties with max/min area

## Task 3:
Join fips code to get full names of states (plesae do a real join using merge)

## Task 4:
Plot maps to show all the counties with the most common names from first task